In [1]:
import pandas as pd
from datetime import datetime, timedelta
csv_file = '../Data_Cleanup/outCSV/Metro_Set_with_IdleTime.csv'
metro = pd.read_csv(csv_file, index_col=0)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
metro['start_time'] = pd.to_datetime(metro['start_time'])

metro ['end_time'] = pd.to_datetime(metro['end_time'])

metro = metro.sort_values(by='start_time')

In [ ]:
#Create the corresponding columns
metro["morgens"] = 0
metro["vormittags"] = 0
metro["mittags"] = 0
metro["nachmittags"]=0
metro["abends"]=0
metro["nachts"]=0

#set the start and end of each day time
morgens = range(5,9)
vormittags = range(9,12)
mittags = range(12,15)
nachmittags = range(15,19)
abends = range(19, 23)
nachts = [23,0,1,2,3,4]

#This list will serve as string provider that we can insert later, to access the proper column
TagesZeiten = ["morgens", "vormittags", "mittags", "nachmittags", "abends", "nachts"]
#This list will serve to provide the start time of each day time, so we know when to change columns
Intervall_starts = [5,9,12,15,19,23]


In [ ]:
metro["monday"] = 0
metro["tuesday"] = 0
metro["wednesday"] = 0
metro["thursday"] = 0
metro["friday"] = 0
metro["saturday"] = 0
metro["sunday"] = 0

Days = ["monday","tuesday","wednesday", "thursday","friday","saturday", "sunday"]

In [ ]:
#Make sure to change the "idle_time" to a timedelta object, so we can use .totalseconds() later
metro["idle_time"] = pd.to_timedelta(metro["idle_time"])

In [ ]:
# Iterate through each row in the DataFrame
for i, row in metro.iterrows():
    end_date = row["end_time"] + row["idle_time"]
    #we only need the starting time hour
    end_hour = end_date.hour
    # and we need to know the duration (idle time) we need to look back
    idle_time = row["idle_time"]
    # make sure to skip any rows with no idle times, since they have 0s in the day times columns
    try:
        NumHours = float(idle_time.total_seconds() / 3600)
        if end_hour in morgens:
            index = 0
        elif end_hour in vormittags:
            index = 1
        elif end_hour in mittags:
            index = 2
        elif end_hour in nachmittags:
            index = 3
        elif end_hour in abends:
            index = 4
        elif end_hour in nachts:
            index = 5

        while NumHours >0 and index > -6:
            metro.at[i, TagesZeiten[index]] = 1
            DistanceToIntervallStart = abs(Intervall_starts[index]- end_hour)

            NumHours -= DistanceToIntervallStart
            end_hour -= DistanceToIntervallStart
            index -= 1

        NumDays = float(row["idle_time"].total_seconds() / (24 * 60 * 60))

        day = end_date.weekday()

        while NumDays > 0 and day > -7:
            metro.at[i, Days[day]] = 1

            NumDays -= 1
            day -= 1
        
    except ValueError:
        continue



In [ ]:
metro.to_csv('../Data_Cleanup/outCSV/Metro_Set_DayAndDayTimes.csv')